# 加入物质对动量角度分辨的影响

In [1]:
import os, sys
import pandas as pd 
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from tqdm import tqdm
from itertools import product

import uproot

this_path = os.path.abspath('.')
if this_path not in sys.path:
    sys.path.append(this_path)

import warnings
warnings.filterwarnings('ignore')

In [2]:
from single_pp_fixed_enery_and_direction_func import *

In [3]:
def readRootFiles(root_files):
    df_list = []
    for root_file in tqdm(root_files):
        df_list.append(readOneFile(root_file))
    df = pd.concat(df_list)

    return df

def readOneFile(root_file):
    rf = uproot.open(root_file)
    tree = rf['RecInfo']

    keys = ['momentum', 'theta', 'phi', 'killed', 'final_rxy', 'final_rz']

    df = pd.DataFrame()
    p_index = [np.argmax(p) if len(p) else None for p in tree['kal_p'].array()]
    for key in keys:
        tmp = tree[key].array()
        if '_p' in key:
            df[key] = [tmp[i][p_index[i]] if not p_index[i] is None else np.nan for i in range(len(tmp)) ] 
        else:
            df[key] = list(tmp)

    df['cos_theta'] = np.cos(df.theta)

    return df

In [4]:
index = ['efficiency', 'hit_additional_layer', 'all',]

def df2Res(df):
    global index
    raw_shape = df.shape[0]
    hit_additional_layer = df[(df.final_rxy>3.37) & (df.final_rxy<5.92) & (df.final_rz<13.4)].shape[0]
    efficiency = hit_additional_layer / raw_shape if raw_shape else 0

    return pd.Series([ efficiency, hit_additional_layer, raw_shape], index=index)



In [7]:
cos_theta_range = np.arange(0, 1, 0.1)
p_range = np.arange(0, 1.3, 0.1)

res_df = pd.DataFrame(columns=['particle', 'material', 'thickness', 'p', 'cos_theta'] + index)

for root_path in tqdm(glob(os.path.join(this_path, '../../data/gen/single_n0/random/*/*'))):
    root_path_copy = root_path
    root_path_split = root_path_copy.replace('\\', '/').split('/')
    particle, material, thickness = root_path_split[-4], root_path_split[-2].split('_')[1], int(root_path_split[-1])

    # print(glob(os.path.join(root_path, '*.root')))
    # continue

    df = readRootFiles(glob(os.path.join(root_path, '*.root')))
    
    for i,j in tqdm(product(range(len(p_range)-1), range(len(cos_theta_range)-1))):
        p1, p2 = p_range[i], p_range[i+1]
        cos_theta1, cos_theta2 = cos_theta_range[j], cos_theta_range[j+1]

        tmp = df.loc[
            (df.momentum >= p1) &
            (df.momentum < p2) &
            (df.cos_theta >= cos_theta1) &
            (df.cos_theta < cos_theta2)
        ]
        res = df2Res(tmp)
        res = [particle, material, thickness, (p1+p2)/2, (cos_theta1+cos_theta2)/2] + list(res)
        res_df.loc[res_df.shape[0]] = res



100%|██████████| 10/10 [00:07<00:00,  1.27it/s]
108it [00:00, 248.35it/s]
100%|██████████| 10/10 [00:08<00:00,  1.23it/s]
108it [00:00, 235.37it/s]
100%|██████████| 10/10 [00:08<00:00,  1.22it/s]
108it [00:00, 229.79it/s]
100%|██████████| 10/10 [00:08<00:00,  1.19it/s]
108it [00:00, 208.79it/s]
100%|██████████| 10/10 [00:08<00:00,  1.23it/s]
108it [00:00, 252.40it/s]
100%|██████████| 10/10 [00:08<00:00,  1.22it/s]
108it [00:00, 243.59it/s]
100%|██████████| 10/10 [00:08<00:00,  1.25it/s]
108it [00:00, 228.88it/s]
100%|██████████| 10/10 [00:08<00:00,  1.16it/s]
108it [00:00, 177.71it/s]
100%|██████████| 10/10 [00:08<00:00,  1.21it/s]
108it [00:00, 241.42it/s]
100%|██████████| 10/10 [00:08<00:00,  1.19it/s]
108it [00:00, 247.48it/s]
100%|██████████| 10/10 [00:08<00:00,  1.19it/s]
108it [00:00, 247.22it/s]
100%|██████████| 10/10 [00:08<00:00,  1.23it/s]
108it [00:00, 243.03it/s]
100%|██████████| 12/12 [01:44<00:00,  8.72s/it]


In [8]:
res_df.to_csv('single_n0.csv')